# Carbon Use Efficiency

Resources:
* Models: http://bigg.ucsd.edu/models/

In [1]:
import requests, cobra, tempfile, os
from cue_tools import loadModelURL, computeCUE

# Load

In [2]:
"""Load model"""
url = 'http://bigg.ucsd.edu/static/models/e_coli_core.mat'
model = loadModelURL(url, 'mat')
model

Name,e_coli_core
Memory address,0x019de3895c48
Number of metabolites,72
Number of reactions,95
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"e, c"


# Compute CUE

In [4]:
"""Compute CUE"""
biomass_rxn = 'BIOMASS_Ecoli_core_w_GAM'
co2_secretion = 'EX_co2_e'

cue = computeCUE(model, biomass_rxn, co2_secretion)
cue

0.038313366655662595